scource code base: https://github.com/rusty1s/pytorch_geometric/blob/master/examples/gat.py

In [1]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch_geometric.datasets import Amazon
import torch_geometric.transforms as T
from torch_geometric.nn import GATConv

import networkx as nx
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
    
def visualize(h, color):
    z = TSNE(n_components=2).fit_transform(h.detach().cpu().numpy())

    plt.figure(figsize=(10,10))
    plt.xticks([])
    plt.yticks([])

    plt.scatter(z[:, 0], z[:, 1], s=70, c=color, cmap="Set2")
    plt.show()

In [2]:
dataset = 'computers'
path = osp.join(osp.dirname(osp.realpath('__file__')), 'data', dataset)
dataset = Amazon(path, dataset, transform=T.NormalizeFeatures())
data = dataset[0]

In [3]:
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Dataset: AmazonComputers():
Number of graphs: 1
Number of features: 767
Number of classes: 10


In [4]:
data = dataset[0]  # Get the first graph object.
num_train_nodes = 84
num_valid_nodes = 500
num_test_nodes = 1000

if not hasattr(data, 'train_mask'):
    # Number of nodes
    num_nodes = data.num_nodes
    
    # Init training, validation and test masks: [False, ..., False]
    train_mask = torch.zeros(num_nodes, dtype=torch.bool)
    val_mask = torch.zeros(num_nodes, dtype=torch.bool)
    test_mask = torch.zeros(num_nodes, dtype=torch.bool)
    
    
    train_index = num_train_nodes
    valid_index = train_index + num_valid_nodes
    test_index = valid_index + num_test_nodes
    
    print(train_index)
    
    train_mask[ :train_index] = True             
    val_mask[train_index:valid_index] = True    
    test_mask[valid_index:test_index] = True      
    
    data.train_mask = train_mask
    data.val_mask = val_mask
    data.test_mask = test_mask
    
    

print(data)

print(data)
print('==============================================================')

# Gather some statistics about the graph.
print(f'Number of nodes: {data.num_nodes}')
print(f'Number of edges: {data.num_edges}')
print(f'Average node degree: {data.num_edges / data.num_nodes:.2f}')
print(f'Number of training nodes: {data.train_mask.sum()}')
print(f'Number of validation nodes: {data.val_mask.sum()}')
print(f'Number of testing nodes: {data.test_mask.sum()}')
print(f'Training node label rate: {int(data.train_mask.sum()) / data.num_nodes:.2f}')
print(f'Contains isolated nodes: {data.contains_isolated_nodes()}')
print(f'Contains self-loops: {data.contains_self_loops()}')
print(f'Is undirected: {data.is_undirected()}')

84
Data(edge_index=[2, 491722], test_mask=[13752], train_mask=[13752], val_mask=[13752], x=[13752, 767], y=[13752])
Data(edge_index=[2, 491722], test_mask=[13752], train_mask=[13752], val_mask=[13752], x=[13752, 767], y=[13752])
Number of nodes: 13752
Number of edges: 491722
Average node degree: 35.76
Number of training nodes: 84
Number of validation nodes: 500
Number of testing nodes: 1000
Training node label rate: 0.01
Contains isolated nodes: True
Contains self-loops: False
Is undirected: True


In [5]:
edge_index = data.edge_index
print(edge_index.t())

tensor([[    0,   507],
        [    0,  6551],
        [    0,  8210],
        ...,
        [13751, 12751],
        [13751, 13019],
        [13751, 13121]])


In [ ]:
visualize(data.x, color=data.y)

In [11]:
num_attention_heads = 8
num_featueres_per_layer =  64
class Net(torch.nn.Module):
    def __init__(self, in_channels, out_channels):
        super(Net, self).__init__()

        self.conv1 = GATConv(in_channels, num_featueres_per_layer, heads=num_attention_heads, dropout=0.6)
        self.conv2 = GATConv(num_featueres_per_layer * num_attention_heads, num_featueres_per_layer, heads=num_attention_heads, dropout=0.6)
        self.conv3 = GATConv(num_featueres_per_layer * num_attention_heads, out_channels, heads=6, concat=False,
                             dropout=0.6)

    def forward(self, x, edge_index):
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv1(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = F.elu(self.conv2(x, edge_index))
        x = F.dropout(x, p=0.6, training=self.training)
        x = self.conv3(x, edge_index)
        return F.log_softmax(x, dim=-1)

In [12]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#device = 'cpu'
model = Net(dataset.num_features, dataset.num_classes).to(device)
data = data.to(device)
labels = data.y.clone().detach().cpu()
optimizer = torch.optim.Adam(model.parameters(), lr=0.005)

In [13]:
def train(data):
    model.train()
    optimizer.zero_grad()
    out = model(data.x, data.edge_index)
    loss = F.nll_loss(out[data.train_mask], data.y[data.train_mask])
    loss.backward()
    optimizer.step()
    return out

In [14]:
@torch.no_grad()
def test(data):
    model.eval()
    out, accs = model(data.x, data.edge_index), []
    for _, mask in data('train_mask', 'val_mask', 'test_mask'):
        acc = float((out[mask].argmax(-1) == data.y[mask]).sum() / mask.sum())
        accs.append(acc)
    return accs

In [15]:
for epoch in range(1, 101):
    out = train(data)
    train_acc, val_acc, test_acc = test(data)
    if epoch % 10 == 0:
        #visualize(out, color=labels)
        print(f'Epoch: {epoch:03d}, Train: {train_acc:.4f}, Val: {val_acc:.4f}, '
          f'Test: {test_acc:.4f}')

Epoch: 010, Train: 0.4048, Val: 0.3780, Test: 0.3940
Epoch: 020, Train: 0.4048, Val: 0.3780, Test: 0.3940
Epoch: 030, Train: 0.4405, Val: 0.3960, Test: 0.4010
Epoch: 040, Train: 0.6429, Val: 0.5780, Test: 0.5680
Epoch: 050, Train: 0.7143, Val: 0.6020, Test: 0.6270
Epoch: 060, Train: 0.7143, Val: 0.6500, Test: 0.6520
Epoch: 070, Train: 0.7619, Val: 0.6700, Test: 0.6820
Epoch: 080, Train: 0.7976, Val: 0.6880, Test: 0.6890
Epoch: 090, Train: 0.8452, Val: 0.6820, Test: 0.7110
Epoch: 100, Train: 0.8571, Val: 0.7180, Test: 0.7260


In [ ]:
out = model(data.x, data.edge_index)
visualize(out, color=labels)